In [17]:
import numpy as np
import pandas as pd
import joblib
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ✅ **Paths to Saved Models**
MODEL_DIR = "../models/"
ARIMA_DIR = os.path.join(MODEL_DIR, "arima_sarima_models")
BOOSTING_DIR = os.path.join(MODEL_DIR, "boosting_models")
RESULTS_FILE = "../data/model_comparison.csv"
FEATURE_DIR = "../data/feature_engineering data"

# ✅ **Function to Load and Validate Data**
def load_stock_data(stock):
    """Loads the feature-engineered dataset for a stock, ensuring 'Date' column exists."""
    file_path = os.path.join(FEATURE_DIR, f"{stock}_boosting_features.csv")

    if not os.path.exists(file_path):
        print(f"⚠️ Feature file missing for {stock}, skipping...")
        return None

    df = pd.read_csv(file_path, parse_dates=["Date"])  # Ensure 'Date' is parsed
    if "Date" not in df.columns:
        print(f"❌ ERROR: 'Date' column not found in {file_path}")
        print(f"🔍 Available columns: {df.columns.tolist()}")
        return None

    df.set_index("Date", inplace=True)
    return df

# ✅ **Function to Load & Predict with Models**
def load_predictions(stock):
    """Loads trained models and makes predictions for a stock."""
    
    df = load_stock_data(stock)
    if df is None:
        return None
    
    last_row = df.iloc[-1].drop("Date", errors="ignore")  # Ensure it doesn't crash if 'Date' is absent

    # ✅ **Load Saved Models (Skipping if Missing)**
    models = {}
    model_paths = {
        "ARIMA": os.path.join(ARIMA_DIR, f"{stock}_ARIMA.pkl"),
        "SARIMA": os.path.join(ARIMA_DIR, f"{stock}_SARIMA.pkl"),
        "Gradient Boosting": os.path.join(BOOSTING_DIR, f"{stock}_GradientBoosting.pkl"),
        "XGBoost": os.path.join(BOOSTING_DIR, f"{stock}_XGBoost.pkl"),
        "LightGBM": os.path.join(BOOSTING_DIR, f"{stock}_LightGBM.pkl"),
    }

    for model_name, path in model_paths.items():
        if os.path.exists(path):
            models[model_name] = joblib.load(path)
        else:
            print(f"⚠️ {model_name} model for {stock} not found, skipping...")

    # ✅ **Make Predictions (Only for Available Models)**
    predictions = {}

    # ✅ **ARIMA/SARIMA Predictions (Use Date Index)**
    for model_name in ["ARIMA", "SARIMA"]:
        if model_name in models:
            try:
                last_index = df.index[-1]
                future_dates = pd.date_range(start=last_index, periods=2, freq="D")[1:]  # Predict next day
                pred_series = models[model_name].predict(start=future_dates[0], end=future_dates[0])
                predictions[model_name] = pred_series.iloc[0]
            except Exception as e:
                print(f"❌ Error in {model_name} prediction for {stock}: {e}")

    # ✅ **Machine Learning Models Predictions**
    for model_name in ["Gradient Boosting", "XGBoost", "LightGBM"]:
        if model_name in models:
            try:
                predictions[model_name] = models[model_name].predict([last_row.values])[0]
            except Exception as e:
                print(f"❌ Error in {model_name} prediction for {stock}: {e}")

    return predictions

# ✅ **Run Evaluation for All Stocks**
if __name__ == "__main__":
    stock_files = [f for f in os.listdir(FEATURE_DIR) if f.endswith("_boosting_features.csv")]
    stock_symbols = [f.split("_")[0] for f in stock_files]

    results = []

    print(f"\n✅ Detected stocks for evaluation: {stock_symbols}")

    for stock in stock_symbols:
        print(f"\n📊 Evaluating models for {stock}...")

        predictions = load_predictions(stock)
        if predictions is None:
            continue  # Skip stock if missing data

        # Get actual last known stock price
        df = load_stock_data(stock)
        y_true = np.array(df[f"{stock}_Close"].iloc[-5:])  # Use last 5 days of actual prices

        # ✅ **Evaluate Each Model**
        for model_name, y_pred in predictions.items():
            results.append(evaluate_model(y_true, np.full_like(y_true, y_pred), model_name, stock))

    # ✅ **Convert Results to DataFrame**
    results_df = pd.DataFrame(results)

    # ✅ **Find the Best Model Per Stock**
    best_models = results_df.loc[results_df.groupby("stock")["rmse"].idxmin()]

    # ✅ **Display & Save Results**
    print("\n📊 Model Performance Comparison:")
    print(results_df)

    print("\n🏆 **Best Model Per Stock:**")
    print(best_models)

    results_df.to_csv(RESULTS_FILE, index=False)
    best_models.to_csv("../data/best_models.csv", index=False)

    print("\n✅ Results saved to 'model_comparison.csv' and 'best_models.csv'")



✅ Detected stocks for evaluation: ['NVDA', 'MSFT', 'TSLA', 'AAPL', 'GOOGL', 'AMZN', 'ORCL', 'IBM', 'META', 'NFLX']

📊 Evaluating models for NVDA...


/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


❌ Error in Gradient Boosting prediction for NVDA: X has 36 features, but GradientBoostingRegressor is expecting 35 features as input.
❌ Error in XGBoost prediction for NVDA: Feature shape mismatch, expected: 35, got 36
❌ Error in LightGBM prediction for NVDA: Number of features of the model must match the input. Model n_features_ is 35 and input n_features is 36

📊 --- ARIMA Evaluation for NVDA ---
✅ RMSE: 1.2027
✅ MAE: 1.1272
✅ MAPE: 7.85%
✅ R² Score: -7.2221
----------------------------------------

📊 --- SARIMA Evaluation for NVDA ---
✅ RMSE: 7.1491
✅ MAE: 7.1368
✅ MAPE: 49.29%
✅ R² Score: -289.4991
----------------------------------------

📊 Evaluating models for MSFT...


/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


❌ Error in Gradient Boosting prediction for MSFT: X has 36 features, but GradientBoostingRegressor is expecting 35 features as input.
❌ Error in XGBoost prediction for MSFT: Feature shape mismatch, expected: 35, got 36
❌ Error in LightGBM prediction for MSFT: Number of features of the model must match the input. Model n_features_ is 35 and input n_features is 36

📊 --- ARIMA Evaluation for MSFT ---
✅ RMSE: 13.5270
✅ MAE: 13.3349
✅ MAPE: 5.61%
✅ R² Score: -34.4512
----------------------------------------

📊 --- SARIMA Evaluation for MSFT ---
✅ RMSE: 65.3165
✅ MAE: 65.2769
✅ MAPE: 27.41%
✅ R² Score: -825.5545
----------------------------------------

📊 Evaluating models for TSLA...


...

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


❌ Error in Gradient Boosting prediction for IBM: X has 36 features, but GradientBoostingRegressor is expecting 35 features as input.
❌ Error in XGBoost prediction for IBM: Feature shape mismatch, expected: 35, got 36
❌ Error in LightGBM prediction for IBM: Number of features of the model must match the input. Model n_features_ is 35 and input n_features is 36

📊 --- ARIMA Evaluation for IBM ---
✅ RMSE: 3.7389
✅ MAE: 3.6523
✅ MAPE: 2.58%
✅ R² Score: -20.8515
----------------------------------------

📊 --- SARIMA Evaluation for IBM ---
✅ RMSE: 1.8558
✅ MAE: 1.6746
✅ MAPE: 1.18%
✅ R² Score: -4.3833
----------------------------------------

📊 Evaluating models for META...


/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


❌ Error in Gradient Boosting prediction for META: X has 36 features, but GradientBoostingRegressor is expecting 35 features as input.
❌ Error in XGBoost prediction for META: Feature shape mismatch, expected: 35, got 36
❌ Error in LightGBM prediction for META: Number of features of the model must match the input. Model n_features_ is 35 and input n_features is 36

📊 --- ARIMA Evaluation for META ---
✅ RMSE: 209.3922
✅ MAE: 209.3839
✅ MAPE: 177.17%
✅ R² Score: -12712.3876
----------------------------------------

📊 --- SARIMA Evaluation for META ---
✅ RMSE: 265.8602
✅ MAE: 265.8537
✅ MAPE: 224.95%
✅ R² Score: -20493.9533
----------------------------------------

📊 Evaluating models for NFLX...
❌ Error in Gradient Boosting prediction for NFLX: X has 36 features, but GradientBoostingRegressor is expecting 35 features as input.
❌ Error in XGBoost prediction for NFLX: Feature shape mismatch, expected: 35, got 36
❌ Error in LightGBM prediction for NFLX: Number of features of the model must ma

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(


In [18]:
import pandas as pd
import os
import joblib
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ✅ **Paths**
MODEL_DIR = "../models/boosting_models/"  # Path where ML models are stored
FEATURE_DIR = "../data/feature_engineering data/"  # Feature dataset path
BEST_MODEL_FILE = "../data/best_models.csv"  # ARIMA & SARIMA best models
COMPARISON_FILE = "../data/model_comparison.csv"  # ARIMA & SARIMA comparison
FINAL_OUTPUT = "../data/full_model_comparison.csv"  # Merged comparison file

# ✅ **Function to Evaluate Model Performance**
def evaluate_model(y_true, y_pred, stock, model_name):
    """Calculate RMSE, MAE, MAPE, and R² Score"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    return {"stock": stock, "model": model_name, "rmse": rmse, "mae": mae, "mape": mape, "r2": r2}

# ✅ **Load ARIMA & SARIMA Results**
arima_sarima_results = pd.read_csv(COMPARISON_FILE) if os.path.exists(COMPARISON_FILE) else pd.DataFrame()

# ✅ **Detect ML Model Files**
ml_models = [f for f in os.listdir(MODEL_DIR) if f.endswith(".pkl")]
stock_symbols = list(set([f.split("_")[0] for f in ml_models]))

# ✅ **Store ML Results**
ml_results = []

# ✅ **Evaluate ML Models for Each Stock**
for stock in stock_symbols:
    print(f"\n📊 Evaluating ML models for {stock}...")

    # ✅ Load Feature Data
    feature_file = os.path.join(FEATURE_DIR, f"{stock}_boosting_features.csv")
    if not os.path.exists(feature_file):
        print(f"❌ Feature file missing for {stock}! Skipping...")
        continue

    df = pd.read_csv(feature_file)
    if "Date" in df.columns:
        df.drop(columns=["Date"], inplace=True)  # Drop Date column if exists

    X_test = df.iloc[-50:, :]  # Use last 50 rows as test data
    y_test = X_test[f"{stock}_Close"].values  # True values
    X_test = X_test.drop(columns=[f"{stock}_Close"])  # Features only

    # ✅ **Load and Evaluate ML Models**
    for model_type in ["GradientBoosting", "XGBoost", "LightGBM"]:
        model_file = os.path.join(MODEL_DIR, f"{stock}_{model_type}.pkl")
        
        if os.path.exists(model_file):
            model = joblib.load(model_file)
            y_pred = model.predict(X_test)
            result = evaluate_model(y_test, y_pred, stock, model_type)
            ml_results.append(result)
        else:
            print(f"❌ Missing {model_type} model for {stock}")

# ✅ **Convert ML Results to DataFrame**
ml_results_df = pd.DataFrame(ml_results)

# ✅ **Merge ARIMA/SARIMA and ML Model Comparisons**
full_comparison_df = pd.concat([arima_sarima_results, ml_results_df])

# ✅ **Find Best Model Per Stock Based on RMSE**
best_models_per_stock = full_comparison_df.loc[full_comparison_df.groupby("stock")["rmse"].idxmin()]

# ✅ **Save Final Results**
full_comparison_df.to_csv(FINAL_OUTPUT, index=False)
best_models_per_stock.to_csv("../data/best_models_per_stock.csv", index=False)

print("\n📊 **Full Model Performance Comparison Saved!**")
print("\n🏆 **Best Model Per Stock (Lowest RMSE):**")
print(best_models_per_stock)



📊 Evaluating ML models for IBM...


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- IBM_Adj Close
Feature names seen at fit time, yet now missing:
- IBM_Close


In [ ]:
import pandas as pd
import os

# ✅ **Define File Paths**
DATA_DIR = "../data"
BEST_MODELS_FILE = os.path.join(DATA_DIR, "best_models.csv")  # ARIMA/SARIMA
ML_MODELS_FILE = os.path.join(DATA_DIR, "model_performance_comparison.csv")  # ML models
COMPARISON_OUTPUT = os.path.join(DATA_DIR, "final_model_comparison.csv")
BEST_MODEL_OUTPUT = os.path.join(DATA_DIR, "final_best_models.csv")

# ✅ **Step 1: Load ARIMA & SARIMA Results**
print("📥 Loading ARIMA & SARIMA results...")
arima_sarima_df = pd.read_csv(BEST_MODELS_FILE)

# ✅ **Step 2: Load ML Model Results**
print("📥 Loading ML Model results...")
ml_df = pd.read_csv(ML_MODELS_FILE)

# ✅ **Step 3: Merge All Models for Comparison**
print("🔄 Merging ARIMA/SARIMA & ML model results...")
comparison_df = pd.concat([arima_sarima_df, ml_df])

# ✅ **Step 4: Find the Best Model per Stock**
best_models_df = comparison_df.groupby("stock").apply(lambda x: x.nsmallest(1, "rmse")).reset_index(drop=True)

# ✅ **Step 5: Save & Display Results**
comparison_df.to_csv(COMPARISON_OUTPUT, index=False)
best_models_df.to_csv(BEST_MODEL_OUTPUT, index=False)

print("\n🏆 **Final Model Comparison Saved:**", COMPARISON_OUTPUT)
print("🏆 **Best Model per Stock Saved:**", BEST_MODEL_OUTPUT)

# ✅ **Step 6: Display Best Models**
print("\n📊 **Best Models for Each Stock**")
print(best_models_df)

# ✅ **Optional: Display First Few Rows of the Full Comparison**
print("\n📊 **Full Model Comparison (First 10 rows)**")
print(comparison_df.head(10))


📥 Loading ARIMA & SARIMA results...
📥 Loading ML Model results...
🔄 Merging ARIMA/SARIMA & ML model results...

🏆 **Final Model Comparison Saved:** ../data/final_model_comparison.csv
🏆 **Best Model per Stock Saved:** ../data/final_best_models.csv


/var/folders/9n/s9t73f3s6lj4xm2bpssc4mw00000gn/T/ipykernel_79900/3897553336.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_models_df = comparison_df.groupby("stock").apply(lambda x: x.nsmallest(1, "rmse")).reset_index(drop=True)


ModuleNotFoundError: No module named 'ace_tools'